# Strategies

In [ ]:
from abc import abstractmethod
from numpy import random
from board import Board
from copy import deepcopy
import concurrent.futures

In [ ]:
class Strategy():
    def __init__(self):
        pass
    @abstractmethod
    def next_move(self, board):
        pass
    def verifyLine(self,board,line):
        freeLines = self.getFreeLines(board)
        for lineTmp in freeLines:
            if (lineTmp[1] == line[0] and lineTmp[2] == line[1]):
                return True
        return False
    
    def getFreeLines(self,board):
        freeLines = []
        for line in board.grid:
            if line[0] == 0:
                freeLines.append(line)
        return freeLines

Strategies is how the game will be played, we created different ones:

In [ ]:
class Human(Strategy):    
    def next_move(self, board):
        chosenLine = input('Enter the line: ')
        if not chosenLine.isdigit():
            print('Invalid input. Please enter a number.')
            return self.next_move(board)
        else:
            if len(chosenLine) == 4:
                chosenLine = (int(chosenLine[0]), int(chosenLine[1])), (int(chosenLine[2]), int(chosenLine[3]))
                verify = self.verifyLine(board, chosenLine)
                if(verify == False):
                    print('Line already taken. Please enter another line.')
                    return self.next_move(board)
                return [0,chosenLine[0],chosenLine[1]]                
            else:
                print('Invalid input. Please enter 4 numbers.')
                return self.next_move(board)
        

Here is the simplest one where the user can play the game, by deciding where to put his lines at every turn

In [ ]:
class Random(Strategy):
    def next_move(self, board):
        freeLines = self.getFreeLines(board)
        if len(freeLines) <= 1:
            return freeLines[0]
        randomChoice = random.randint(0, len(freeLines) - 1)
        return freeLines[randomChoice]
        

Random is just used to test and plays a random free line at every turn, to test the algorithms efficiencies

In [ ]:
class Greedy(Strategy):
    def __init__(self,player):
        self.player = player
    def next_move(self, board):
        boardTmp = deepcopy(board)
        scoreTmp = 100000
        freeLines = self.getFreeLines(boardTmp)
        for line in freeLines:
            boardTmp = deepcopy(board)
            boardTmp.make_move(line, self.player)
            if boardTmp.get_score1(line,self.player) < scoreTmp:
                scoreTmp = boardTmp.get_score(0)
                chosenLine = line
                
        return chosenLine

Greedy is a type of algorithm that puts a line wherever it can score a point, if it can t score a point it just put a line in the first space avaiable

In [ ]:
class AlphaBeta(Strategy):
    def __init__(self, depth,player):
        self.depth = depth
        self.player = player
    def next_move(self, board):
        freeLines = self.getFreeLines(board)
        possibileEnd = False
        if len(freeLines) <= self.depth:
            possibileEnd = True
        if self.player == 1:
            results = self.alpha_beta(board, self.depth, -100000, 100000, True, freeLines)
        else:
            results = self.alpha_beta(board, self.depth, -100000, 100000, False, freeLines)
        text_file = open("AlphaBeta.txt", "w")
        txt = results[2]
        txt += "\nFinal decision:\nChosenLine " + str(results[1]) + "\n" + "Eval " + str(results[0]) + "\n"
        if possibileEnd:
            txt += "Possibile end of game:\n"
            if results[0] == 0:
                txt += "It's a draw!\n"
            elif results[0] > 0:
                txt += "Player 2 wins!\n"
            else:
                txt += "Player 1 wins!\n"
        text_file.write(txt)
        text_file.close()
        return results[1]
    
    def alpha_beta(self, board, depth, alpha, beta, maximizingPlayer, freeLines):
        chosenLine = None
        log = ""
        if depth == 0 or board.is_full():
            return  board.get_score(0), chosenLine, ""
        if maximizingPlayer:         
            maxEval = -100000
            scoreTmp = [board.get_score_old(),board.get_score_old()]
            for line in freeLines:
                boardTmp = deepcopy(board)
                boardTmp.make_move(line, 1)
                scoreTmp[0] = boardTmp.get_score1(line,1)
                if scoreTmp[0] == scoreTmp[1]:
                    if depth != 1:
                        #indexTmp = freeLines.index(line)
                        #freeLineTmp = freeLines.pop(indexTmp)
                        results = self.alpha_beta(boardTmp, depth-1, alpha, beta, False, self.getFreeLines(boardTmp))
                        eval = results[0]
                        log += results[2]
                        #freeLines.insert(indexTmp,freeLineTmp)
                    else:
                        eval = scoreTmp[0]
                else:
                    scoreTmp[1] = scoreTmp[0]
                    #indexTmp = freeLines.index(line)
                    #freeLineTmp = freeLines.pop(indexTmp)
                    results = self.alpha_beta(boardTmp, depth - 1, alpha, beta, True, self.getFreeLines(boardTmp))
                    eval = results[0]
                    log += results[2]
                    #freeLines.insert(indexTmp,freeLineTmp)
                    #sprint("eval ",eval)
                if eval > maxEval:
                    #print("Maxeval ",eval, " ChosenLine ",line)
                    log += "\nMaxeval " + str(eval) + " ChosenLine " + str(line) + "\n"
                    #log += boardTmp.__str__()
                    chosenLine = line
                    maxEval = eval
                if alpha <= eval:
                    alpha = eval
                    if beta <= alpha:
                        break
            return maxEval, chosenLine , log
        else:
            minEval = 100000
            scoreTmp = [board.get_score_old(),board.get_score_old()]
            for line in freeLines:  
                boardTmp = deepcopy(board)          
                boardTmp.make_move(line, -1)
                scoreTmp[0] = boardTmp.get_score1(line,-1)
                if scoreTmp[0] == scoreTmp[1]:
                    if depth != 1:
                        #indexTmp = freeLines.index(line)
                        #freeLineTmp = freeLines.pop(indexTmp)
                        results = self.alpha_beta(boardTmp, depth-1, alpha, beta, True, self.getFreeLines(boardTmp))
                        eval = results[0]
                        log += results[2]
                        #freeLines.insert(indexTmp,freeLineTmp)
                    else:
                        eval = scoreTmp[0]
                else:
                    scoreTmp[1] = scoreTmp[0]
                    #indexTmp = freeLines.index(line)
                    #freeLineTmp = freeLines.pop(indexTmp)
                    results = self.alpha_beta(boardTmp, depth -1, alpha, beta, False, self.getFreeLines(boardTmp))
                    eval = results[0]
                    log += results[2]
                    #freeLines.insert(indexTmp,freeLineTmp)
                if eval < minEval:
                    log += "\nMineval " + str(eval) + " ChosenLine " + str(line) + "\n"
                    #log += boardTmp.__str__()
                    #print("Mineval ",eval, " ChosenLine ",line)
                    chosenLine = line
                    minEval = eval
                if beta >= eval:
                    beta = eval
                    if beta <= alpha:
                        break
            return minEval,chosenLine , log

Finally we have alpha beta algorythm

Here's a breakdown of the main components of the algorithm:

Initialization:
The AlphaBeta class is initialized with a specified depth and player.
depth: The depth of the search tree, determining how many moves ahead the algorithm explores.
player: The current player (1 or -1).
next_move Method:
Takes the current game board as input and calculates the next move for the player using the Alpha-Beta algorithm.
It checks if the number of free lines on the board is less than or equal to the specified depth. If true, it sets possibileEnd to True.
Calls the alpha_beta function to find the best move.
Writes the results to a file named "AlphaBeta.txt" and returns the chosen move.
alpha_beta Method:
Recursive function implementing the Alpha-Beta algorithm.
Parameters:
board: The current state of the game board.
depth: The remaining depth to explore in the game tree.
alpha and beta: Parameters for pruning the search space.
maximizingPlayer: A boolean indicating whether the current player is maximizing.
freeLines: The available lines to make a move.
Returns a tuple containing the evaluation score of the current move, the chosen line, and a log string.
Details:
The algorithm uses a depth-first search approach to explore possible moves.
It utilizes the getFreeLines and make_move methods on the board object.
The evaluation function is based on the get_score1 method of the board object.
Pruning is performed using the alpha-beta values to discard unnecessary branches in the search tree.
The results and additional information are logged into the "AlphaBeta.txt" file.

at every iteration it adds a line to the log that it's printed at the end of the research and shows every intermediate result and at the end specifies the chosen move with its maximum/minimum results and specifies if a player can win and with what score